In [1]:
from src.utils.coordinates_util import get_coordinates_for_city, haversine_distance
from src.scrapers.scrapers import BasicScraper
from src.types.types import Coordinates, URL, XPath

import pandas as pd

In [2]:
def assert_gps_coordinates_equal(
    coord1: Coordinates, coord2: Coordinates, tolerance: float = 1e-2
):
    assert (
        abs(coord1.latitude - coord2.latitude) < tolerance
    ), f"Latitude difference exceeds tolerance: {coord1.latitude} vs {coord2.latitude}"
    assert (
        abs(coord1.longitude - coord2.longitude) < tolerance
    ), f"Longitude difference exceeds tolerance: {coord1.longitude} vs {coord2.longitude}"


expected_coordinates: Coordinates = Coordinates(48.89452, 18.04436)
actual_coordinates: Coordinates = get_coordinates_for_city("Trenčín")

assert_gps_coordinates_equal(actual_coordinates, expected_coordinates)

In [3]:
get_coordinates_for_city("Ul. gen. M. R. Štefánika 426, 911 01 Trenčín")

Coordinates not found for Ul. gen. M. R. Štefánika 426, 911 01 Trenčín


In [4]:
get_coordinates_for_city("gen. M. R. Štefánika 426, 911 01 Trenčín")

Coordinates(latitude=48.8906828, longitude=18.0566796)

In [5]:
city1: str = "Bratislava"
city2: str = "Košice"

coord1: Coordinates = get_coordinates_for_city(city1)
coord2: Coordinates = get_coordinates_for_city(city2)

if coord1 and coord2:
    distance = haversine_distance(coord1, coord2)
    print(f"The distance between {city1} and {city2} is {distance:.2f} kilometers.")
else:
    print("Unable to get coordinates for one or both cities.")

The distance between Bratislava and Košice is 309.44 kilometers.


In [6]:
df_cities = pd.read_csv("data/obyvatelia_na_obec.csv")

assert df_cities.shape[0] == 2927, "Number of municipalities doesn't match"
df_cities.head(7)

,Kód,Územná jednotka,Spolu,muži (abs.),muži (%),ženy (abs.),ženy (%)
0,SK0101528595,Bratislava - mestská časť Staré Mesto,46080,22210,48.20,23870,51.80
1,SK0102529311,Bratislava - mestská časť Podunajské Biskupice,23464,11193,47.70,12271,52.30
2,SK0102529320,Bratislava - mestská časť Ružinov,81004,37575,46.39,43429,53.61
3,SK0102529338,Bratislava - mestská časť Vrakuňa,20711,9945,48.02,10766,51.98
4,SK0103529346,Bratislava - mestská časť Nové Mesto,44458,20981,47.19,23477,52.81
5,SK0103529354,Bratislava - mestská časť Rača,25733,12717,49.42,13016,50.58
6,SK0103529362,Bratislava - mestská časť Vajnory,6079,3056,50.27,3023,49.73
